In [8]:
pwd

u'/Users/naveenkumar/Dropbox (Personal)/OnlineWork/Notebooks'

Delcaring all Libraries

In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

import warnings


Setting Parameters

In [10]:
sns.set(style='white', context='notebook', palette='deep')
warnings.filterwarnings('ignore')

Important functions

In [11]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

In [12]:
def roundn(x,n):
    from math import pow
    return round(x*pow(10,n))/pow(10,n)

def columntype(x):
    t = ''
    if is_numeric_dtype(x) == True:
        t = 'numeric'
    else:
        t = 'string'
    
    return t

def unique_value_counts(x,y,cut_off):
    if y <= cut_off:
        return (s2[x].value_counts()).to_dict()
    
    
def replace_DDMMYY_WITH_DDMMYYYY(indate, AddCentury):
    if len(str(indate)) == 8:
        YY = str(indate)[6:8]
        YY_INT = int(YY) + AddCentury
        return str(indate)[0:6] + str(YY_INT)
    else:
        return np.nan
    

def datediff(d1,d2,date_format):
    df = date_format
    a = datetime.strptime(str(d1), df)
    b = datetime.strptime(str(d2), df)
    delta = b - a
    return delta.days



        
        
def ds_summary(s2):
    cmd = pd.DataFrame(s2.columns,columns=['column_name'])
    cmd['TotalRows'] = len(s2)
    cmd['TotalUniqueLevels'] = cmd['column_name'].apply(lambda col:s2[col].nunique())
    cmd['PercentUniqueValues'] = cmd['TotalUniqueLevels']/cmd['TotalRows']
    cmd['TotalNullValues'] = cmd['column_name'].apply(lambda col:s2[col].isnull().sum())
    cmd['PercentNullValues'] = cmd['TotalNullValues']/cmd['TotalRows']
    cmd['IdColFlag'] = cmd['TotalRows'] == cmd['TotalUniqueLevels']
    cmd['DataType'] = cmd['column_name'].apply(lambda col:columntype(s2[col]))
    cmd['UniqueValue'] = cmd[cmd['TotalUniqueLevels'] < 11]['column_name'].apply(lambda col:(s2[col].value_counts()).to_dict())
    cmd['mean'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].mean(),2))
    cmd['min'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].min(),2))
    cmd['q1'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.25),2))
    cmd['median'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].median(),2))
    cmd['q3'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.75),2))
    cmd['max'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].max(),2))
    return cmd


In [13]:
# Calculate information value
def calc_iv(df, feature, target, pr=0):

    lst = []

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Bad'])
    data = data[data['Bad'] > 0]

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    data['ValIV'] = (data['WoE'] * (data['Distribution Good'] - data['Distribution Bad']))

    data = data.sort_values(by=['Variable', 'Value'], ascending=True)

    IV = data['ValIV'].sum()
    
    if pr == 1:
        print(data)
    
    return IV

In [ ]:
def compute_iv_ds(df_in, target_col, n_quantile=5):
    
    y_d = d[target_col].to_frame()
    
    iv_df = pd.DataFrame()
    
    f_list = df_in.columns.tolist()
    f_list.remove(target_col)
    
    
    
    for f in f_list:
        f_dtype = df_in[f].dtype.name
        
        if df_in[f].max() > df_in[f].min():
            if re.sub(r'(float|int)[0-9]+', r'float', f_dtype) == 'float':
        
                tmp_df = pd.qcut(d[f], n_quantile, labels=None, retbins=False, precision=3, duplicates='drop').to_frame()
                tmp_df = tmp_df.join(y_d)

                iv_out = np.round(calc_iv(tmp_df, f, target_col),2)
                
                tmp_iv_df = pd.DataFrame({'feature' : [f],
                                          'datatype' :[d[f].dtype],
                                           'iv' : [iv_out]
                                         })
                iv_df = iv_df.append(tmp_iv_df)

    
            elif re.sub(r'.*bool.*', r'bool', f_dtype) == 'bool':

                    tmp_df = d[f].to_frame()
                    tmp_df = tmp_df.join(y_d)
                    iv_out = np.round(calc_iv(tmp_df, f, target_col),2)

                    tmp_iv_df = pd.DataFrame({'feature'  : [f],
                                              'datatype' :[d[f].dtype],
                                               'iv'      : [iv_out]
                                             })

                    iv_df = iv_df.append(tmp_iv_df)

            else :
                print('IV computattion of '+ f_dtype + ' is under progess')

    return iv_df 

In [14]:
def percentile_dist(x):
    print("-----------------------------------")
    for t in np.arange(0.01,1.0,0.01):
        print(str(t) + " : " + str(x.quantile(q=t)))

Directories and Files

In [15]:
inputdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/Titanic/input/'
outputdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/Titanic/output/'

trainf = 'train.csv'
testf = 'test.csv'

trainfpath = inputdir + trainf
testfpath = inputdir + testf

In [17]:
train = pd.read_csv(trainfpath)
test = pd.read_csv(testfpath)

In [20]:
t = train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
